## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key) 

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head() 

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.10,-79.24,35.01,86,90,8.05,overcast clouds
1,1,Portland,US,45.52,-122.68,44.01,100,75,9.17,broken clouds
2,2,Hasaki,JP,35.73,140.83,32.00,55,75,1.12,broken clouds
3,3,Castro,BR,-24.79,-50.01,71.10,76,54,6.33,broken clouds
4,4,Chimore,BO,-16.98,-65.13,90.18,48,66,4.07,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10) 

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Castro,BR,-24.79,-50.01,71.10,76,54,6.33,broken clouds
10,10,Tual,ID,-5.67,132.75,84.79,77,100,14.90,overcast clouds
11,11,Rikitea,PF,-23.12,-134.97,78.13,72,17,7.16,few clouds
16,16,Kruisfontein,ZA,-34.00,24.73,76.60,67,0,11.48,clear sky
17,17,Pochutla,MX,15.74,-96.47,82.99,65,5,4.68,clear sky
19,19,Brazzaville,CG,-4.27,15.28,84.20,62,20,5.82,few clouds
22,22,Georgetown,MY,5.41,100.34,78.01,94,40,6.78,heavy intensity rain
23,23,Saint-Philippe,RE,-21.36,55.77,78.80,69,0,25.28,clear sky
29,29,Rahuri,IN,19.38,74.65,75.29,46,5,6.11,clear sky
30,30,El Alto,PE,-4.27,-81.21,75.20,69,40,17.22,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count() 

268


Unnamed: 0             268
City                   268
Country                266
Lat                    268
Lng                    268
Max Temp               268
Humidity               268
Cloudiness             268
Wind Speed             268
Current Description    268
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head() 

266


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Castro,BR,-24.79,-50.01,71.10,76,54,6.33,broken clouds
10,10,Tual,ID,-5.67,132.75,84.79,77,100,14.90,overcast clouds
11,11,Rikitea,PF,-23.12,-134.97,78.13,72,17,7.16,few clouds
16,16,Kruisfontein,ZA,-34.00,24.73,76.60,67,0,11.48,clear sky
17,17,Pochutla,MX,15.74,-96.47,82.99,65,5,4.68,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Castro,BR,71.10,broken clouds,-24.79,-50.01,
10,Tual,ID,84.79,overcast clouds,-5.67,132.75,
11,Rikitea,PF,78.13,few clouds,-23.12,-134.97,
16,Kruisfontein,ZA,76.60,clear sky,-34.00,24.73,
17,Pochutla,MX,82.99,clear sky,15.74,-96.47,
19,Brazzaville,CG,84.20,few clouds,-4.27,15.28,
22,Georgetown,MY,78.01,heavy intensity rain,5.41,100.34,
23,Saint-Philippe,RE,78.80,clear sky,-21.36,55.77,
29,Rahuri,IN,75.29,clear sky,19.38,74.65,
30,El Alto,PE,75.20,scattered clouds,-4.27,-81.21,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key 
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Castro,BR,71.10,broken clouds,-24.79,-50.01,CHACARA BAILLY
10,Tual,ID,84.79,overcast clouds,-5.67,132.75,Grand Vilia Hotel
11,Rikitea,PF,78.13,few clouds,-23.12,-134.97,Pension Maro'i
16,Kruisfontein,ZA,76.60,clear sky,-34.00,24.73,Oyster Bay House Rental
17,Pochutla,MX,82.99,clear sky,15.74,-96.47,Hotel Posada San Jose
19,Brazzaville,CG,84.20,few clouds,-4.27,15.28,Hotel Murex
22,Georgetown,MY,78.01,heavy intensity rain,5.41,100.34,Cititel Penang
23,Saint-Philippe,RE,78.80,clear sky,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
29,Rahuri,IN,75.29,clear sky,19.38,74.65,P.W.D RESTHOUSE
30,El Alto,PE,75.20,scattered clouds,-4.27,-81.21,Black Marlin Lodge & Restaurant


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))